In [1]:
import time, cv2, torch, threading, argparse, os
import torch.backends.cudnn as cudnn
import numpy as np 
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms

from models.experimental import attempt_load
from utils.augmentations import letterbox
from utils.general import check_img_size, non_max_suppression, scale_coords 
from utils.accessory_lib import system_info
from PIPNet.networks import Pip_resnet101
from PIPNet.functions import forward_pip, get_meanface
from scipy.spatial import distance
from tqdm import tqdm

In [2]:
bus_img_dir = "C:\\workspace\\image\\bus\\"
passenger_img_dir = "C:\\workspace\\image\\passenger\\"
taxi_img_dir = "C:\\workspace\\image\\taxi\\"
truck_img_dir = "C:\\workspace\\image\\truck\\"

In [3]:
bus_img_path_list = []
for sub_dir_name in os.listdir(bus_img_dir):
    for img_name in os.listdir(bus_img_dir+sub_dir_name):
        bus_img_path_list.append(bus_img_dir+sub_dir_name+os.sep+img_name)

bus_img_path_list

['C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_01.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_02.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_03.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_04.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_05.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_06.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_07.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_08.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_09.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_10.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_11.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_12.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M_01_M0_G1_C0_13.jpg',
 'C:\\workspace\\image\\bus\\R_216_60_M\\R_216_60_M

In [4]:
net_stride = 32
num_nb = 10
data_name = 'data_300W'
experiment_name = 'pip_32_16_60_r101_l2_l1_10_1_nb10'
num_lms = 68
face_landmark_input_size = 240
det_box_scale = 1.2
eye_det = 0.12

img_size = 640
CONF_THRES = 0.4
IOU_THRES = 0.45
half = True
cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = False

elapsed_time: float = 0.0
fps: float = 0.0
ref_frame: int = 0
det_frame: int = 0
prev_time = time.time()
start_time = time.time()

device = torch.device("cuda")

meanface_indices, reverse_index1, reverse_index2, max_len = get_meanface(os.path.join('PIPNet', 'data', data_name,
                                                                                      'meanface.txt'), num_nb)

resnet101 = models.resnet101(weights='ResNet101_Weights.DEFAULT')
face_landmark_net = Pip_resnet101(resnet101, num_nb=num_nb, num_lms=num_lms, input_size=face_landmark_input_size,
                                  net_stride=net_stride)
face_landmark_net = face_landmark_net.to(device)

face_landmark_weight_file_path = os.path.join('PIPNet', 'snapshots', data_name, experiment_name, f'epoch{60-1}.pth')
face_landmark_net.load_state_dict(torch.load(f=face_landmark_weight_file_path, map_location=device))
face_landmark_net.eval()

face_landmark_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
preprocess = transforms.Compose([transforms.ToPILImage(),
                                 transforms.Resize((face_landmark_input_size, face_landmark_input_size)),
                                 transforms.ToTensor(), face_landmark_normalize])

face_detector_weight_file_path = os.path.join('.', 'weights', 'face_detection_yolov5s.pt')
face_det_normalize_tensor = torch.tensor(255.0).to(device)

system_info()

# Initialize
print(f'[1/3] Device Initialized {time.time() - prev_time:.2f}sec')


 ----- Environment Status -----
Operating System Type: ('64bit', 'WindowsPE')
Python Version: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Pytorch Version: 1.13.1+cu116
Open Computer Vision Version: 4.8.1
GPU Available: True
GPU Device Name: NVIDIA GeForce RTX 3080 Ti Laptop GPU
Number of GPU: 1
Tensor Core TF32 Enable: False 

[1/3] Device Initialized 3.34sec


In [5]:
# Load model
model = attempt_load(face_detector_weight_file_path, device)  # load FP32 model
model.eval()
stride = int(model.stride.max())  # model stride
img_size_chk = check_img_size(img_size, s=stride)  # check img_size

if half:
    model.half()  # to FP16

# Get names and colors
names = model.module.names if hasattr(model, 'module') else model.names

# Run inference
model(torch.zeros(1, 3, img_size_chk, img_size_chk).to(device).type_as(next(model.parameters())))  # run once
print(f'[2/3] Yolov5 Detector Model Loaded {time.time() - prev_time:.2f}sec')

Fusing layers... 
newYOLOv5s summary: 224 layers, 7053910 parameters, 0 gradients


[2/3] Yolov5 Detector Model Loaded 6.67sec


In [16]:
with torch.no_grad():
    global elapsed_time, fps, ref_frame, det_frame, prev_time

    for img_name in tqdm(bus_img_path_list):
        img0 = cv2.imread(img_name)
        ref_frame += 1
        #print(img0.shape)

        # Padded resize
        img = letterbox(img0, img_size_chk, stride=stride)[0]

        # Convert
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img = torch.divide(img.to(device), face_det_normalize_tensor)
        img = img.unsqueeze(0)

        # Inference
        pred = model(img, augment=False)[0]

        # Apply NMS
        detected_face = non_max_suppression(pred, CONF_THRES, IOU_THRES, classes=None, agnostic=False)[0]

        # Process detections
        if len(detected_face):
            # Rescale boxes from img_size to img0 size
            detected_face[:, :4] = scale_coords(img.shape[2:], detected_face[:, :4], img0.shape).round()

            # Write results
            for *xyxy, conf, cls in reversed(detected_face):
                det_xmin = int(xyxy[0])
                det_ymin = int(xyxy[1])
                det_xmax = int(xyxy[2])
                det_ymax = int(xyxy[3])
                det_width = det_xmax - det_xmin
                det_height = det_ymax - det_ymin
                
                det_xmin -= int(det_width * (det_box_scale - 1) / 2)
                det_ymin -= int(det_height * (det_box_scale - 1) / 2)
                det_xmax += int(det_width * (det_box_scale - 1) / 2)
                det_ymax += int(det_height * (det_box_scale - 1) / 2)

                det_xmin = max(det_xmin, 0)
                det_ymin = max(det_ymin, 0)
                det_xmax = min(det_xmax, img0.shape[0] - 1)
                det_ymax = min(det_ymax, img0.shape[1] - 1)

                det_width = det_xmax - det_xmin + 1
                det_height = det_ymax - det_ymin + 1
                det_crop = img0[det_ymin:det_ymax, det_xmin:det_xmax, :]
                
                if det_crop.shape[0] > 0 and det_crop.shape[1] > 0:
                    det_crop = cv2.resize(src=det_crop, dsize=(face_landmark_input_size, face_landmark_input_size),
                                          interpolation=cv2.INTER_AREA)
                    cv2.rectangle(img0, (det_xmin, det_ymin), (det_xmax, det_ymax), (0, 0, 255), 1)

                    inputs = preprocess(det_crop).to(device)
                    inputs = inputs.unsqueeze(0)
                    lms_pred_x, lms_pred_y, lms_pred_nb_x, lms_pred_nb_y, outputs_cls, max_cls = forward_pip(face_landmark_net,
                                                                                                             inputs, preprocess,
                                                                                                             face_landmark_input_size,
                                                                                                             net_stride, num_nb)

                    lms_pred = torch.cat((lms_pred_x, lms_pred_y), dim=1).flatten()
                    tmp_nb_x = lms_pred_nb_x[reverse_index1, reverse_index2].view(num_lms, max_len)
                    tmp_nb_y = lms_pred_nb_y[reverse_index1, reverse_index2].view(num_lms, max_len)
                    tmp_x = torch.mean(torch.cat((lms_pred_x, tmp_nb_x), dim=1), dim=1).view(-1, 1)
                    tmp_y = torch.mean(torch.cat((lms_pred_y, tmp_nb_y), dim=1), dim=1).view(-1, 1)

                    lms_pred_merge = torch.cat((tmp_x, tmp_y), dim=1).flatten()
                    lms_pred = lms_pred.cpu().numpy()
                    lms_pred_merge = lms_pred_merge.cpu().numpy()

                    eye_x = (lms_pred_merge[36 * 2:48 * 2:2] * det_width).astype(np.int32) + det_xmin
                    eye_y = (lms_pred_merge[(36 * 2) + 1:(48 * 2) + 1:2] * det_height).astype(np.int32) + det_ymin

                    left_eye_horizontal_dist = distance.euclidean((eye_x[0], eye_y[0]), (eye_x[3], eye_y[3]))
                    left_eye_vertical_dist = min(distance.euclidean((eye_x[1], eye_y[1]), (eye_x[5], eye_y[5])),
                                                 distance.euclidean((eye_x[2], eye_y[2]), (eye_x[4], eye_y[4])))

                    right_eye_horizontal_dist = distance.euclidean((eye_x[6], eye_y[6]), (eye_x[9], eye_y[9]))
                    right_eye_vertical_dist = min(distance.euclidean((eye_x[11], eye_y[11]), (eye_x[7], eye_y[7])),
                                                  distance.euclidean((eye_x[10], eye_y[10]), (eye_x[8], eye_y[8])))

                    left_eye_ratio = left_eye_vertical_dist / left_eye_horizontal_dist
                    right_eye_ratio = right_eye_vertical_dist / right_eye_horizontal_dist

 

                    if left_eye_ratio < eye_det and right_eye_ratio < eye_det:
                        detection_state = 1
                    else:
                        detection_state = 0


        print(img_name, left_eye_ratio, right_eye_ratio, left_eye_ratio > eye_det, right_eye_ratio > eye_det)
        fps = 1/(time.time() - prev_time)
        prev_time = time.time()
        # Stream results
        elapsed_time = time.time()-start_time

  0%|          | 2/8067 [00:00<14:36,  9.21it/s]

C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_01.jpg 0.034088708186481805 0.5423261445466404 False True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_02.jpg 0.07048063607217736 0.24325212770525997 False True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_03.jpg 0.042835293687811936 0.25300956667772434 False True


  0%|          | 8/8067 [00:00<06:19, 21.26it/s]

C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_04.jpg 0.20203050891044216 0.6293167755275527 True True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_05.jpg 0.04753289820861187 0.22360679774997896 False True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_06.jpg 0.06775963568181181 0.4152273992686998 False True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_07.jpg 0.2165561168014839 0.8514693182963201 True True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_08.jpg 0.11277677488692019 0.12403473458920847 False True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_09.jpg 0.07453559924999299 1.414213562373095 False True
C:\workspace\image\bus\R_216_60_M\R_216_60_M_01_M0_G1_C0_10.jpg 0.11467769256492184 0.10699941519444955 False False


  0%|          | 16/8067 [00:00<04:43, 28.37it/s]

...

  3%|▎         | 254/8067 [00:07<03:40, 35.37it/s]

C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_09.jpg 0.13895262654079812 0.12572074400897873 True True
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_10.jpg 0.08598671607846956 0.09765286720945168 False False
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_11.jpg 0.1212678125181665 0.04081802660877408 True False
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_12.jpg 0.17402385355726993 0.055006532413609206 True False
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_13.jpg 0.052374288905649574 0.09478247207640289 False False
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_14.jpg 0.0540106735637485 0.1124251093148722 False False
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_15.jpg 0.06449003705885216 0.16786822525727785 False True


  3%|▎         | 257/8067 [00:07<03:48, 34.22it/s]

C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_16.jpg 0.15420415139834365 0.11061326375099442 True False
C:\workspace\image\bus\R_217_40_M\R_217_40_M_01_M0_G1_C0_17.jpg 0.14332591242820383 0.09198870231668353 True False


KeyboardInterrupt: 